# **Data Cleaning Notebook**

## Objectives

* Evaluate and manage the missing data
* Clean data
        
## Inputs
        
* outputs/datasets/collection/HousePricesData.csv
        
## Outputs
        
* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned
* Data cleaning pipeline

## Additional comments 

* We will only be dealing with the HousePricesData dataset as the inherited houses dataset does not require analysis, it will only be used to make predictions.


---

## Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Load the data

* Import the pandas library
* Load the dataset as a pandas DataFrame and assign it to our dataframe df_prices
* View the data in the df_prices variable

In [ ]:
import pandas as pd
df_prices = (pd.read_csv("outputs/datasets/collection/HousePricesData.csv"))
df_prices.head(5)

---

## Data exploration

In this section we are interested in delving deeper into the database. We want to identify the missing data, check the distribution and shape of the variables linked to said missing data. 

* First, we will identify the variables containing missing data:
    * We create a list containing all columns with missing data
    * We check it's length
    * We show the variables containing missing data
    * We show the count and type of missing data for each variable

In [ ]:
vars_with_missing_data = df_prices.columns[df_prices.isna().sum() > 0].to_list()
len(vars_with_missing_data)

In [ ]:
vars_with_missing_data

In [ ]:
df_prices[vars_with_missing_data].info()

* We will now use panda's profiling report to get an overview of the missing data as well as detailed information for each variable. Here we are only considering the variables containing missing data, **not** all variables of our dataset.
    * We notice that 26% of our data is missing
    * We have both numerical and categorical data missing
    * In the "alerts" section, we notice that two variables have very high levels of missing data. They are "EnclosedPorch" and "WoodDeckSF" with 90.7% and 89.4% of missing data respectively.
    * In the same section, we see that the variables "2ndFlrSF" and "MasVnrArea" have 53.5% and 59% of zeros respectively.

In [ ]:
from pandas_profiling import ProfileReport
pandas_report = ProfileReport(df=df_prices[vars_with_missing_data], minimal=True)
pandas_report.to_notebook_iframe()

---

## Data cleaning

### Preparation

We will create a custom function to visualize missing data levels. It will show the data type, how many rows are missing and what percentage it represents.

In [ ]:
def VisualizeMissingData(df):
  """
  Function to visualize missing values levels within dataframe
  """
  missing_data_absolute = df.isnull().sum()
  missing_data_percentage = round(missing_data_absolute/len(df)*100 , 2)
  df_missing_data = (pd.DataFrame(
                          data= {"DataType":df.dtypes,
                                 "RowsWithMissingData": missing_data_absolute,
                                 "PercentageOfDataset": missing_data_percentage}
                                  )
                    .sort_values(by=['PercentageOfDataset'],ascending=False)
                    .query("PercentageOfDataset > 0")
                    )

  return df_missing_data

We can apply this to our data set to check it works by comparing the results with the information given by panda's profile report.

In [ ]:
VisualizeMissingData(df_prices)

We will use a custom function presented in Feature Engine Unit 9 of the course in order to assess the effect of cleaning the data. This function can be used when the input mean, median or arbitrary number is a numerical variable and/or when we replace with 'Missing' or most frequent a categorical variable. The parameters are:
        * df_original: data not cleaned
        * df_cleaned: cleaned data
        * variables_applied_with_method: variables where you applied a given method

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

def DataCleaningEffect(df_original,df_cleaned,variables_applied_with_method):

  flag_count=1 # Indicate plot number
  
  # distinguish between numerical and categorical variables
  categorical_variables = df_original.select_dtypes(exclude=['number']).columns 

  # scan over variables, 
    # first on variables that you applied the method
    # if the variable is a numerical plot, a histogram if categorical plot a barplot
  for set_of_variables in [variables_applied_with_method]:
    print("\n=====================================================================================")
    print(f"* Distribution Effect Analysis After Data Cleaning Method in the following variables:")
    print(f"{set_of_variables} \n\n")
  

    for var in set_of_variables:
      if var in categorical_variables:  # it is categorical variable: barplot
        
        df1 = pd.DataFrame({"Type":"Original","Value":df_original[var]})
        df2 = pd.DataFrame({"Type":"Cleaned","Value":df_cleaned[var]})
        dfAux = pd.concat([df1, df2], axis=0)
        fig , axes = plt.subplots(figsize=(15, 5))
        sns.countplot(hue='Type', data=dfAux, x="Value",palette=['#432371',"#FAAE7B"])
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.xticks(rotation=90)
        plt.legend() 

      else: # it is numerical variable: histogram

        fig , axes = plt.subplots(figsize=(10, 5))
        sns.histplot(data=df_original, x=var, color="#432371", label='Original', kde=True,element="step", ax=axes)
        sns.histplot(data=df_cleaned, x=var, color="#FAAE7B", label='Cleaned', kde=True,element="step", ax=axes)
        axes.set(title=f"Distribution Plot {flag_count}: {var}")
        plt.legend() 

      plt.show()
      flag_count+= 1

### Split data into train and test sets

First we will split our data into train and test sets. This will allow us to see the results of our imputations on the test set after training on the train set.

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df_prices,
                                        df_prices['SalePrice'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

We can check our missing data levels in our train set are similar to the ones present in our full dataset. 

In [ ]:
df_missing_data_trainset = VisualizeMissingData(TrainSet)
print(f"* There are {df_missing_data_trainset.shape[0]} variables with missing data \n")
df_missing_data_trainset

Now that we have explored our missing data in more detail, we can assess how we will deal with it.

### Variables with high levels of missing data

* We will make the decision to drop the two variables with high levels of missing data ("EnclosedPorch" and "WoodDeckSF", with more than 89% of data missing). Such a high level of missing data means that the variables will not have any noticeable influence on our target variable. We will therefore save time by removing them from our cleaned data set early.

In [ ]:
from feature_engine.selection import DropFeatures
imputer = DropFeatures(features_to_drop=['EnclosedPorch' , 'WoodDeckSF'])
imputer.fit(TrainSet)
df_method = imputer.transform(TrainSet)

* We can check the effects of our imputation.
    * We notice that the two features with more than 89% data missing have been dropped.

In [ ]:
df_missing_data_trainset = VisualizeMissingData(df_method)
print(f"* There are {df_missing_data_trainset.shape[0]} variables with missing data \n")
df_missing_data_trainset

### Categorical data

* We have two categorical variables that contain missing data. GarageFinish, representing the interior finish of the garage, with 11.22% of missing data. And BsmtFinType1, representing the rating of the basement finished area, with 7.62% of missing data.
* We can deal with them in a few different ways:
    * Delete the whole column. This would mean loosing a lot of data as we are only missing about 10% of our data and we could loose valuable insight.
    * Replace the missing values with the most frecuent ones. This could be useful but further imbalance our dataset.
    * Replace the missing values with the one of our choice.
    * Delete rows with null values. This runs the risk of reducing our dataset
    * Develop a model to predict the missing values.
* Deleting the whole column would make us loose possible valuable insights as about 90% of our data is present. Replacing the missing values with the most frecuent ones risks imbalancing our dataset and affecting the influence of that category too much. Deleting rows with missing values will affect the overall size of our dataset. Developing a model would be the most accurate option, we might look into this option in further iterations of this project.
* In this scenario we will choose the 3d option. We will use the categorical imputation to assign the category "None" to all missing data. This will allow us to quickly deal with low levels of missing data whilst not infuencing the correlation between existing data and the target variable. We will then use our DataCleaningEffect function to visualize the results of our imputation.

In [ ]:
from feature_engine.imputation import CategoricalImputer

variables_categorical = ['GarageFinish' , 'BsmtFinType1']
imputer = CategoricalImputer(imputation_method='missing', fill_value='None', variables=variables_categorical)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_categorical)

### Numerical data

* We have five numerical variables with missing data:
    * LotFrontage, representing the linear feet of street connected to the property , with 18.15% of missing data.
    * BedroomAbvGr, representing the bedrooms above grade (not including basement bedrooms) , with 6.85% of missing data.
    * 2ndFlrSF, representing the second floor square footage, with 5.14% of missing data.
    * GarageYrBlt, representing the year the garage was built, with 4.97% of missing data.
    * MasVnrArea, representing the Masonry veneer area in square feet, with 0.51% of missing data.
* As we are dealing with numerical values, we will use a Mean Median Imputer to replace missing data by the mean or median value of the variable. In order to assess if we will choose the mean or the median, we will see if the variables' distribution is normal.

In [ ]:
for col in ['LotFrontage' , 'BedroomAbvGr', '2ndFlrSF', 'GarageYrBlt', 'MasVnrArea']:
  sns.histplot(data=TrainSet, x=col, kde=True)
  plt.show()

* As we can see from these histogram plots, LotFrontage and BedroomAbvGr have a bell shaped distribution which indicates they are normally distributed and we can use mean imputation.
* The rest of the variables, 2ndFlrSF, GarageYrBlt adn MasVnrArea are skewed, either to the right or to the left. We will therefore use the median as the mean is affected by outlier values and may not be as representative of our data.

#### Mean imputation and visualisation

In [ ]:
from feature_engine.imputation import MeanMedianImputer

variables_mean = ['LotFrontage' , 'BedroomAbvGr']
imputer = MeanMedianImputer(imputation_method='mean', variables=variables_mean)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_mean)

#### Median imputation and visualisation

In [ ]:
variables_median = ['2ndFlrSF' , 'GarageYrBlt', 'MasVnrArea']
imputer = MeanMedianImputer(imputation_method='median', variables=variables_median)
df_method = imputer.fit_transform(TrainSet)
DataCleaningEffect(df_original=TrainSet,
                   df_cleaned=df_method,
                   variables_applied_with_method=variables_median)

---

## Arrange cleaning steps in pipeline

* We will arrange our data cleaning steps in a pipeline. This will facilitate the data cleaning of our whole dataset.
* The steps are as follows:
    * Dropping EnclosedPorch and WoodDeckSF variables.
    * Categorical imputation on GarageFinish and BsmtFinType1 variables.
    * Numerical mean imputation on LotFrontage and BedroomAbvGr variables.
    * Numerical median imputation on 2ndFlrSF, GarageYrBlt and MasVnrArea variables.

In [ ]:
from sklearn.pipeline import Pipeline

dataCleaning_pipeline = Pipeline([
    
      ( 'drop',  DropFeatures(features_to_drop=['EnclosedPorch', 'WoodDeckSF']) ),
      ( 'categorical',  CategoricalImputer(imputation_method='missing',
                                     fill_value='None',
                                     variables=['GarageFinish' , 'BsmtFinType1']) ),
      ( 'mean',  MeanMedianImputer(imputation_method='mean',
                                     variables=['LotFrontage' , 'BedroomAbvGr']) ),
      ( 'median',  MeanMedianImputer(imputation_method='median',
                                     variables=['2ndFlrSF', 'GarageYrBlt', 'MasVnrArea']) )
      
])

* We apply run the whole dataset through the pipeline to clean all our data

In [ ]:
TrainSet, TestSet = dataCleaning_pipeline.fit_transform(TrainSet) , dataCleaning_pipeline.fit_transform(TestSet)
df_prices = dataCleaning_pipeline.fit_transform(df_prices)

* We confirm that there is no missing data left in our Train and Test set as well as our dataframe.

In [ ]:
VisualizeMissingData(TrainSet)

In [ ]:
VisualizeMissingData(TestSet)

In [ ]:
VisualizeMissingData(df_prices)

---

# Push files to Repo

* We push our Train, Test and cleaned data sets to the repo

In [ ]:
try:
  os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
  print(e)

TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)
df_prices.to_csv("outputs/datasets/cleaned/HousePricesCleaned.csv", index=False)


* And we save our pipeline in case of further use (such as cleaning the inherited houses data).

In [ ]:
import joblib

file_path = f'outputs/ml_pipeline/data_cleaning'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

In [ ]:
joblib.dump(value=dataCleaning_pipeline, filename=f"{file_path}/dataCleaning_pipeline.pkl")